In [ ]:
# This cell does not get executed when run with Times Square
import os
import datetime

day_obs = int(os.environ.get("SCHEDVIEW_DAY_OBS", datetime.date.today().strftime("%Y%m%d")))
earliest_sim = datetime.date.fromisoformat(os.environ.get("SCHEDVIEW_SIM_DATE", datetime.date(day_obs//10000, (day_obs%10000)//100, day_obs%100).isoformat()))

In [ ]:
day_obs = '2025-10-08'
earliest_sim = datetime.date.fromisoformat('2025-01-01')

In [ ]:
import os
import sys

#sched_source = 'usdf'
#sched_source = 'shared'
sched_source = 'devel'
match sched_source:
    case 'shared':
        if os.path.exists('/sdf/data/rubin/shared/scheduler/packages'):
            sys.path.insert(0, "/sdf/data/rubin/shared/scheduler/packages/uranography-1.2.1")
            sys.path.insert(0, "/sdf/data/rubin/shared/scheduler/packages/rubin_scheduler-3.11.0")
            sys.path.insert(0, "/sdf/data/rubin/shared/scheduler/packages/rubin_sim-2.2.4")
            sys.path.insert(0, "/sdf/data/rubin/shared/scheduler/packages/schedview-0.18.1.dev27+g2dced97")
    case 'devel':
        if os.path.exists('/sdf/data/rubin/user/neilsen/devel'):
            sys.path.insert(0, "/sdf/data/rubin/user/neilsen/devel/uranography")
            sys.path.insert(0, "/sdf/data/rubin/user/neilsen/devel/rubin_scheduler")
            sys.path.insert(0, "/sdf/data/rubin/user/neilsen/devel/rubin_sim")
            sys.path.insert(0, "/sdf/data/rubin/user/neilsen/devel/schedview")
    case _:
        # Use the current environment
        pass

In [ ]:
from schedview.util import config_logging_for_reports
import logging
config_logging_for_reports(logging.ERROR)

In [ ]:
from IPython.display import display, HTML, Markdown
from collections import OrderedDict
import datetime
import pandas as pd
import numpy as np
import astropy
import bokeh
import bokeh.io
import boto3
import warnings
import astropy.units as u
from astropy.time import Time, TimezoneInfo
from urllib.parse import urlparse
from lsst.resources import ResourcePath
import yaml
from contextlib import redirect_stdout
import io
from erfa import ErfaWarning

In [ ]:
os.environ['RUBIN_SIM_DATA_DIR'] = '/sdf/data/rubin/shared/rubin_sim_data'

In [ ]:
import schedview.collect.resources
import schedview.plot
import rubin_sim
from rubin_sim.sim_archive import vseqarchive
from schedview import DayObs

In [ ]:
# Suppress benign warnings.

warnings.filterwarnings(
    "ignore",
    category=ErfaWarning,
    message=r".*ERFA function.*dubious year.*",
)

In [ ]:
metadata_dsn = {
    'host': 'usdf-maf-visit-seq-archive-tx-ro.sdf.slac.stanford.edu',
    'user': 'tester',
    'database': 'opsim_log',
}
metadata_schema = 'test'
visit_seq_archive_metadata = vseqarchive.VisitSequenceArchiveMetadata(metadata_dsn, metadata_schema)

In [ ]:
max_age = (datetime.date.today() - earliest_sim).days + 1

In [ ]:
def make_prenight_link(row):
    url = f"https://usdf-rsp.slac.stanford.edu/times-square/github/lsst/schedview_notebooks/prenight/prenight?day_obs={day_obs}&sim_date={row.sim_creation_day_obs}&sim_index={row.daily_id}"
    return f'<a href={url} target="_blank" rel="noopener noreferrer">{row.visitseq_label}</a>'

def display_prenight_html_for_telescope(this_telescope_prenights):
    prenight_html = (
        this_telescope_prenights
        .assign(simulation=this_telescope_prenights.apply(make_prenight_link, axis='columns'))
        .loc[:, ['sim_creation_day_obs', 'daily_id', 'simulation', 'first_day_obs', 'last_day_obs', 'scheduler_version', 'tags']]
        .rename(columns={
            'sim_creation_day_obs': 'Date simulation was run',
            'daily_id': 'id',
        })
    ).to_html(escape=False)
    display(HTML(prenight_html))

# Pre-night simulation archive overview with simulations including the `day_obs` of {{ params.day_obs }}

In [ ]:
display(Markdown(f"Including simulations recorded on or after {earliest_sim}."))

### Simonyi telescope

In [ ]:
simonyi_prenights = visit_seq_archive_metadata.sims_on_nights(DayObs(day_obs).date, DayObs(day_obs).date, tags=["prenight"], telescope='simonyi', max_simulation_age=max_age).set_index('visitseq_uuid')
display_prenight_html_for_telescope(simonyi_prenights)

### AuxTel

In [ ]:
auxtel_prenights = visit_seq_archive_metadata.sims_on_nights(DayObs(day_obs).date, DayObs(day_obs).date, tags=["prenight"], telescope='auxtel', max_simulation_age=max_age).set_index('visitseq_uuid')
display_prenight_html_for_telescope(auxtel_prenights)

In [ ]:
os.environ['AWS_PROFILE'] = 'prenight'

In [ ]:
from lsst.resources import ResourcePath

In [ ]:
rp = ResourcePath('s3://rubin:rubin-scheduler-prenight/opsim/test/prenight_index/auxtel/2025/10/auxtel_prenights_for_2025-10-08.json')

In [ ]:
with rp.as_local() as local_rp:
    df = pd.read_json(local_rp.ospath, orient='index')

In [ ]:
df